In [ ]:
# %%time
# %%capture
# !apt-get update
# !apt-get install texlive texlive-extra-utils

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/bin/lin

In [ ]:
!make4ht -v


make4ht version v0.1c


In [ ]:
# htlatex filename.tex "xhtml,html5,charset=utf-8" " -cmozhtf -utf8"
# make4ht -u filename.tex

In [ ]:
drive_save_path = "/content/drive/My Drive/Colab Notebooks/next_sentence/english_dataset/"
# drive_save_path = "/mnt/e/next_sentence/english_dataset/"


Command to work with Google Drive

Install required packages

In [ ]:
!pip3 install requests beautifulsoup4

In [ ]:
!python --version

Python 3.8.5


In [ ]:
!pip3 install html5lib

# 1. Arxiv Crawler

Import packages

In [ ]:
import requests
from bs4 import BeautifulSoup
import tarfile
import urllib.request
import os
import subprocess
import re
import traceback
import io
import json
import logging

Crawler code

In [ ]:
if not os.path.exists("temp"):
  os.mkdir("temp")

def fetch_paper(source_id):
  try:
    source_url = "https://export.arxiv.org/e-print/"+source_id
    logging.info(source_url)
    source_archive = io.BytesIO(urllib.request.urlopen(source_url).read())
    tar = tarfile.open(fileobj=source_archive,mode='r:gz')
    tex_counter = 0
    sub_tex_counter = 0
    # extracting_members = []
    main_tex_member = ""
    for member in tar:
      filename = member.name.lower()
      if(filename.endswith('.tex')):
        # logging.info(member.name)
        # extracting_members.append(member.name)
        if("/" not in filename):
          tex_counter+=1
          main_tex_member=member.name;
        else:
          sub_tex_counter+=1
          # logging.info(tar.extractfile(member).read())#.decode('utf-8'))
    if(tex_counter>1):
      logging.info("more than 1 tex at root, skipped...")
      source_archive.close()
      return None;
    # create temp folder
    if not os.path.exists("temp/"+source_id):
      os.mkdir("temp/"+source_id)
    # remove comments and dump all tex file to temp folder
    # logging.info("extracting... "+str(extracting_members))


    # change directory, run detex and get result
    path = os.path.abspath("temp/"+source_id+"/")
    tar.extractall(path=path)
    command = "make4ht -u "+main_tex_member
    # logging.info(path)
    # logging.info(command)
    try:
        text = subprocess.check_output(command,cwd=path, shell=True, timeout=10)

    except subprocess.CalledProcessError as e:
        print(e.output.decode('utf-8'))
        return None

    output_file = main_tex_member.replace(".tex", ".html")
    with open("temp/"+source_id+"/"+output_file, encoding='utf-8') as f:
        html_document = f.read()
        soup = BeautifulSoup(html_document, 'html5lib')
        paragraphs = soup.find("body").find_all("p", recursive=False)
        paragraphs = [p.text.strip() for p in paragraphs if p.text.strip()]

    return paragraphs

  except Exception as e:
    logging.info(e)
    traceback.print_exc()
    return None


In [ ]:
def process_page(year, month, page, per_page):
  page_result = {
      "results":[],
      "first_index":""
  }
  try:
    URL = 'https://export.arxiv.org/list/cs/{:0>2d}{:0>2d}?skip={}&show={}'.format(year,month,page*per_page,per_page)
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')

    data_list = soup.select('dl')[0]
    source_link_tags = data_list.findChildren("dt", recursice=False)
    source_data = data_list.findChildren("dd", recursive=False)
    if(len(source_link_tags)==0):
      return None
    if(len(source_link_tags)!=len(source_data)):
      logging.info("list length not match")
      return None
    else:
      for index in range(len(source_link_tags)):
        try:
          if page_result['first_index']=="":
            page_result['first_index'] = source_link_tags[index].select('a')[0].getText()
          other_format_tag = source_link_tags[index].select('a[title="Other formats"]')
          journal_ref_tag = source_data[index].select('div.list-journal-ref')
          name=source_data[index].select('div.list-title')[0].getText()
          authors=[a.getText() for a in source_data[index].select('div.list-authors a')]

          if len(other_format_tag)==0:
            # logging.info(index, "No source")
            continue
          if len(journal_ref_tag)==0:
            # logging.info(index, "Not published")
            continue
          source_id = other_format_tag[0]['href'].replace('/format/','')
          logging.info(source_id)
          paragraphs = fetch_paper(source_id)
          if paragraphs is None:
            # logging.info(index, " Unable to extract ",source_id)
            continue
          result = {
              "id":source_id,
              "name":name,
              "journal":journal_ref_tag[0].getText(),
              "authors":authors,
              "paragraphs":paragraphs
          }
          page_result['results'].append(result)
          logging.info("                         Satistied")
        except Exception as e:
          traceback.print_exc()
          print(e)
  except Exception as e:
    traceback.print_exc()
    print(e)
  return page_result

In [ ]:
if not os.path.exists(drive_save_path):
    os.makedirs(drive_save_path, )
logging.basicConfig(filename=drive_save_path+"log.txt",
                            filemode='a',
                            format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                            datefmt='%H:%M:%S',
                            level=logging.DEBUG)

In [ ]:
if not os.path.exists(drive_save_path+"result"):
  os.mkdir(drive_save_path+"result")
year = 19
per_page = 100
logger = logging.getLogger()
for month in range(7,13):
  page = 0
  if month==7 and year==19:
      page+=18
  first_index=""
  while True:
    logging.info("Processing: "+"{}-{}-page {}".format(year, month, page))
    page_result = process_page(year,month,page,per_page)
    logging.info(page_result['first_index'])
    if page_result['first_index']==first_index:
      break;
    else:
      first_index=page_result['first_index']
    if page_result is not None:
      for result in page_result['results']:
        with open(drive_save_path+"result/"+result['id']+".json", 'w', encoding='utf-8') as output_file:
          json.dump(result,output_file, indent=4, ensure_ascii=False)
      page+=1
    else:
      break
    logger.handlers[0].flush()

Output dir: 	
Compiler: 	latex
Latex options: 	 -jobname=conference_041818 
tex4ht.sty :	xhtml,,charset=utf-8
tex4ht	 -cmozhtf -utf8
build_file	conference_041818.mk4
Cannot open config file	conference_041818.mk4
setting param correct_exit
setting param correct_exit
setting param correct_exit
setting param ext
Adding: 	ext	dvi
LaTeX call: latex  -jobname=conference_041818  '\makeatletter\def\HCode{\futurelet\HCode\HChar}\def\HChar{\ifx"\HCode\def\HCode"##1"{\Link##1}\expandafter\HCode\else\expandafter\Link\fi}\def\Link#1.a.b.c.{\g@addto@macro\@documentclasshook{\RequirePackage[#1,html]{tex4ht}}\let\HCode\documentstyle\def\documentstyle{\let\documentstyle\HCode\expandafter\def\csname tex4ht\endcsname{#1,html}\def\HCode####1{\documentstyle[tex4ht,}\@ifnextchar[{\HCode}{\documentstyle[tex4ht]}}}\makeatother\HCode xhtml,,charset=utf-8.a.b.c.\input conference_041818.tex'
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=latex)
 restricted \write18 enable

KeyboardInterrupt: ignored

Crawling done

## 1.2 Fix from temp folder

In [ ]:
from os import listdir, mkdir
from os.path import isfile, join, exists
from bs4 import BeautifulSoup
import json

temp_folder = "/mnt/e/temp/"
save_folder = "/mnt/e/arxiv_dataset/"
if not exists(save_folder):
    mkdir(save_folder)
dirs = listdir(temp_folder)
print(len(dirs))
print(dirs[0])
for folder in dirs:
    files = [t for t in listdir(temp_folder+folder) if isfile(temp_folder+folder+"/"+t)]
    tex = [t for t in files if t.endswith('.tex')]
    print(files)
    if len(tex)==0:
        continue
    tex = tex[0]
    html_file = tex.replace('.tex','.html')
    try:
        with open(temp_folder+folder+"/"+html_file) as f:
            soup=BeautifulSoup(f, "html5lib")
            paragraphs = soup.find('body').find_all('p', recursive=False)
            paragraphs = [p.text.strip() for p in paragraphs if p.text.strip()]
            with open(save_folder+folder+".json", 'w', encoding='utf-8') as output_file:
                result = {
                "id":folder,
                "paragraphs":paragraphs
                }
                json.dump(result,output_file, indent=4, ensure_ascii=False)
        print(tex)
    except:
        pass

Output hidden; open in https://colab.research.google.com to view.


# 2. Clean up and compile dataset


In [ ]:
from os import listdir
from os.path import isfile, join
import json
import re
from IPython.display import HTML, display

def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

To check the result

In [ ]:
path = drive_save_path+"arxiv_dataset/"
files = [f for f in listdir(path) if isfile(join(path, f))]
print("Total paper collected: ",len(files))

with open(join(path,files[8])) as f:
  data = f.read()
  print(len(data))
  print(data)
  # data = json.load(f)


Total paper collected:  1581
53410
{
    "id": "1901.00741",
    "paragraphs": [
        "Modeling, Analysis, and Mitigation of Dynamic\nBotnet Formation in Wireless IoT Networks",
        "Muhammad Junaid Farooq, Student Member, IEEE\nand Quanyan Zhu, Member, IEEE",
        "Muhammad Junaid Farooq and Quanyan Zhu are with the Department of Electrical & Computer\nEngineering, Tandon School of Engineering, New York University, Brooklyn, NY, USA, E-mails:\n{mjf514, qz494}@nyu.edu.",
        "Abstract\n       The  Internet  of  Things  (IoT)  relies  heavily  on  wireless  communication  devices  that  are  able  to  discover  and  interact  with\n    other  wireless  devices  in  their  vicinity.  The  communication  flexibility  coupled  with  software  vulnerabilities  in  devices,  due  to\n    low  cost  and  short  time-to-market,  exposes  them  to  a  high  risk  of  malware  infiltration.  Malware  may  infect  a  large  number  of\n    network devices using device-to-device (D2D

Fix crawled result
- If in the result paragraph exists "\t" then split to 2 paragraphs
- Remove email
- Tab. -> Table     Eq. -> Equation
- Fix titles and journal-ref

In [ ]:
def further_clean_up(data):
    # paragraphs = []
    #   for para in data['paragraphs']:
    new_paras = [re.sub(r'(\d+)\.(\d+)','\1\2',p) for p in data['paragraphs']]
    new_paras = [re.sub(r"(?<!\w)([A-Za-z])\.", r"\1", p.replace('\n',' ')) for p in new_paras]
    new_paras = [re.sub(r'\[(\d*)\]',r'\1',p) for p in new_paras]
    new_paras = [re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b','',p) for p in new_paras] # email
    new_paras = [p.replace("Tab.", "Table ").replace("Eq.", "Equation ").replace("Fig.", "Figure ") for p in new_paras] #tab -> table...
    new_paras = [p.replace("tab.", "Table ").replace("eq.", "Equation ").replace("fig.", "Figure ") for p in new_paras] #tab -> table...
    new_paras = [re.sub(r"\s\s+" , " ", p) for p in new_paras]
    new_paras = [p for p in new_paras if len(p)<5000 and len(p)>20]
    new_paras = [p for p in new_paras if p[0].isupper()]
      
    # paragraphs.extend(new_paras)
    data['paragraphs'] = new_paras
    #   data['name'] = data['name'].replace("Title: ","",1).replace("\n","")
    #   data['journal'] = data['journal'].replace("Journal-ref: ","",1).replace("\n","")
    return data

In [ ]:
with open(join(path,files[4])) as f:
  data = json.load(f)
  print(json.dumps(further_clean_up(data), indent=4))

{
    "id": "1901.00306",
    "paragraphs": [
        "Recommender systems aim to predict the probability that a specific user will like a specific resource. Therefore, recommender systems utilize the past user behavior (eg, resources previously consumed by this user) in order to generate a personalized list of potentially relevant resources 32. Popular application domains of recommender systems include online marketplaces (eg, Amazon and Zalando), movie and music streaming services (eg, Netflix and Spotify), job portals (eg, LinkedIn and Xing), and social tagging systems (eg, BibSonomy and CiteULike). ______________________________________ Permission to make digital or hard copies of all or part of this work for personal or classroom use is granted without fee provided that copies are not made or distributed for profit or commercial advantage and that copies bear this notice and the full citation on the first page. Copyrights for components of this work owned by others than ACM must b

Combine into 1 large json file (raw dataset)


In [ ]:
path = drive_save_path+"arxiv_dataset/"
files = [f for f in listdir(path) if isfile(join(path, f))]
total_files = len(files)
print("Total paper collected: ",total_files)

out = display(progress(0, total_files), display_id=True)

final_list = []
count = 0
for file in files:
  count+=1
  out.update(progress(count, total_files))
  with open(join(path,file), encoding='utf-8') as f:
    data = json.load(f)
    final_list.append(further_clean_up(data))

Total paper collected:  1581


In [ ]:
result_path = drive_save_path+"english_dataset_full.json"
with open(result_path, "w") as f:
  json.dump(final_list, f)

Combine into 1 large json file with filtered information (task specific dataset)

In [ ]:
out = display(progress(0, total_files), display_id=True)

filtered_list = []
for entry in final_list:
  count+=1
  out.update(progress(count, total_files))
  # TODO: remove short paragraphs
  filtered_paragraphs = []
  for para in entry['paragraphs']:
    sentences = para.split('.')
    sentences = [s.strip() for s in sentences if len(s.strip())>40]
    if len(sentences)>=3:
      filtered_paragraphs.append(sentences)
  filtered_list.append({
      "id":entry['id'],
      "paragraphs":filtered_paragraphs
  })

In [ ]:
result_path = drive_save_path+"english_dataset_filtered.json"
with open(result_path, "w", encoding='utf-8') as f:
  json.dump(filtered_list, f, ensure_ascii=False)
samples_path = drive_save_path+"english_dataset_sample.json"
with open(samples_path, "w") as f:
  json.dump(filtered_list[:2], f, indent=4, ensure_ascii=False)

In [1]:
import json
drive_save_path = "/content/drive/My Drive/Colab Notebooks/next_sentence/english_dataset/"
with open(drive_save_path+"english_dataset_filtered_v2.json", encoding='utf-8') as filtered_file:
    data = json.load(filtered_file)


In [2]:
print(len(data))

1581


In [3]:
test = data[:158]
train = data[158:]
with open(drive_save_path+"english_dataset_filtered_v2_train.json", 'w', encoding='utf-8') as f:
    json.dump(train, f, ensure_ascii=False)
with open(drive_save_path+"english_dataset_filtered_v2_test.json", 'w', encoding='utf-8') as f:
    json.dump(test, f, indent=4, ensure_ascii=False)

In [7]:
groups = [len(entry['paragraphs']) for entry in train]
print(sum(groups))

40302


Total number of satisfied paragraphs

In [ ]:
groups = [len(entry['paragraphs']) for entry in filtered_list]
print(sum(groups))

45051


In [ ]:
print(json.dumps(final_list[8], indent=4))
print(json.dumps(filtered_list[8], indent=4))

{
    "id": "1901.00741",
    "paragraphs": [
        "Modeling, Analysis, and Mitigation of Dynamic Botnet Formation in Wireless IoT Networks",
        "Muhammad Junaid Farooq, Student Member, IEEE and Quanyan Zhu, Member, IEEE",
        "Muhammad Junaid Farooq and Quanyan Zhu are with the Department of Electrical & Computer Engineering, Tandon School of Engineering, New York University, Brooklyn, NY, USA, E-mails: {mjf514, qz494}@nyu.edu.",
        "Abstract The Internet of Things (IoT) relies heavily on wireless communication devices that are able to discover and interact with other wireless devices in their vicinity. The communication flexibility coupled with software vulnerabilities in devices, due to low cost and short time-to-market, exposes them to a high risk of malware infiltration. Malware may infect a large number of network devices using device-to-device (D2D) communication resulting in the formation of a botnet, ie, a network of infected devices controlled by a common mal